In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from sklearn.preprocessing import normalize
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import xgboost
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import math
from sklearn import preprocessing as PP
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [ ]:
#Reading the CSV Datafile using Pandas
path_of_file='/content/drive/MyDrive/IIIT/Projects/Activity Recognition Model/dsads_preprocessed_data - dsads_preprocessed_data.csv'
dataset=pd.read_csv(path_of_file)
dataset.head()
#Building a classification model to predict the activity based on the 405 features given in the dataset

features=dataset.drop(['Seconds','Activities','Subject'],axis=1)
activities=dataset['Activities']
subjects=dataset.iloc[:,-1:]
print(activities.unique())

#Splitting the training and test dataset shape
X_train,X_test,Y_train,Y_test = train_test_split(features,activities,random_state=421212, test_size=0.2,shuffle=True)
X_train=np.asarray(X_train)
Y_train=np.asarray(Y_train)
X_test=np.asarray(X_test)
Y_test=np.asarray(Y_test)
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32)
Train_loader = DataLoader(list(zip(X_train, Y_train)), shuffle=True, batch_size=4)
Test_loader = DataLoader(list(zip(X_test, Y_test)), shuffle=False, batch_size=2)

Training_loss=[]                                                                # List to store the Training loss values
Validation_loss=[]                                                              # List to store the Validation loss values
Training_accuracy=[]                                                            # List to store the Training Accuracy values
Validation_accuracy=[]                                                          # List to store the Validation Accuracy values

# Define the MultiLayer Perceptron model
class MLPClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 200)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(200, 100)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(100, 50)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(50, number_of_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x

# Function to calculate accuracy
def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# Function to calculate precision, recall, and f1-score
def precision_recall(model, criterion, optimizer, Test_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, labels in Test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted',labels=np.unique(y_pred))
    return precision, recall, f1


# Function to train the model
def train(model, criterion, optimizer, Train_loader):
    model.train()                                                               # Model is set to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in Train_loader:
        optimizer.zero_grad()                                                   # Making the gradients zero before passing the next batch
        labels =  labels.type(torch.LongTensor)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()                                                         # Calculates the gradient value based on loss for all parameters/weights
        optimizer.step()                                                        # Updates the parameters based on the calculated gradients
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(Train_loader)
    epoch_accuracy = 100 * correct / total
    Training_loss.append(epoch_loss)
    Training_accuracy.append(epoch_accuracy)
    return epoch_loss, epoch_accuracy

# Function to validate the model
def validate(model, criterion, Test_loader):
    model.eval()                                                                # Model is set to validating/testing/evaluating mode
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in Test_loader:
            labels =  labels.type(torch.LongTensor)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(Test_loader)
    epoch_accuracy = 100 * correct / total
    Validation_loss.append(epoch_loss)
    Validation_accuracy.append(epoch_accuracy)
    return epoch_loss, epoch_accuracy

# Define hyperparameters
input_size = 405                                                                # Enter the input size (number of features)                                                                 # Enter the number of hidden units
number_of_classes = 20                                                          # Number of classes in the dataset
lr = 0.001                                                                      # Learning rate
num_epochs = 20                                                                 # Number of Epochs

# Initialize the model, loss function, and optimizer
model = MLPClassifier(input_size, number_of_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, criterion, optimizer, Train_loader)
    val_loss, val_accuracy = validate(model, criterion, Test_loader)
    precision, recall, _ = precision_recall(model, criterion, optimizer, Test_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, "
          f"Precision: {precision:.4f}, Recall: {recall:.4f}")

# Calculate and print confusion matrix
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in Test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

fig, ax = plt.subplots()
line_up, = ax.plot(Training_loss, label='Training Loss')
line_down, = ax.plot(Validation_loss, label='Validation Loss')
line_side, = ax.plot(Training_accuracy, label='Training Accuracy')
line_sides, = ax.plot(Validation_accuracy, label='Validation Accuracy')
ax.legend(handles=[line_up, line_down, line_side, line_sides])

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,...,Feature_399,Feature_400,Feature_401,Feature_402,Feature_403,Feature_404,Feature_405,Seconds,Activities,Subject
0,0.182,0.001290,0.977,0.001200,0.920,0.002120,0.604,0.182,0.001110,0.001040,...,0.756,0.0783,0.00880,0.000857,0.000414,0.243,0.04750,1.0,1.0,1.0
1,0.182,0.000961,0.979,0.000642,0.942,0.001490,0.573,0.182,0.000792,0.000812,...,0.752,0.0793,0.00701,0.001380,0.000779,0.467,0.27200,2.0,1.0,1.0
2,0.182,0.000528,0.991,0.000466,0.937,0.000539,0.615,0.182,0.000407,0.000345,...,0.747,0.0746,0.00528,0.001150,0.000368,0.184,0.02850,3.0,1.0,1.0
3,0.182,0.001530,0.984,0.001470,0.938,0.001970,0.583,0.182,0.001250,0.001170,...,0.696,0.0872,0.00942,0.000855,0.000282,0.188,0.02740,4.0,1.0,1.0
4,0.181,0.000815,0.989,0.000720,0.937,0.000945,0.542,0.181,0.000586,0.000560,...,0.723,0.0906,0.00718,0.000837,0.000469,0.173,0.00997,5.0,1.0,1.0


In [1]:
# # importing the zipfile module
# from zipfile import ZipFile

# # loading the temp.zip and creating a zip object
# with ZipFile("/content/drive/MyDrive/IIIT/Projects/Activity Recognition Model/daily+and+sports+activities.zip", 'r') as zObject:

# 	# Extracting specific file in the zip
# 	# into a specific location.
# 	zObject.extractall(path="/content/drive/MyDrive/IIIT/Projects/Activity Recognition Model")
# zObject.close()